# Najprostszy model

##Wstęp
Piosenki nie mają przypisanych gatunków, musimy temu zarządzić. Ponieważ wcześniej tych danych nie mieliśmy, nie możemy ocenić, jak dobrze dobrany jest gatunek piosenki.
Jakość podziału na grupy oceniamy na podstawie danych, które uzyskaliśmy: ile gatunków występuje w otrzymanym zbiore, jaka jest liczba piosenk poszcególnego gatunku. Liczba tych gatunków musi być zbliżona do liczby najczęściej występujących, "głównych" gatunków, również staramy się uzyskać bardziej zbalansowany podział na gatunki, staramy się unikać pojawiania się grup o pewnym gatunku, do którego należą 1-2 piosenki.


In [16]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
from matplotlib import pyplot

In [17]:
!pip install pandas==1.5.2
!pip install joblib==1.2.0
from joblib import load, dump

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
tracks = load("/content/tracks_df_preprocessed(1)")

In [19]:
tracks.head(2)

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre
0,48SFtLr5URCI97X2Ynfdnc,Par Avion (Live) ( 2014 - Remaster) - Live; 20...,0,291227,0,2yTUYhIf8fxptTIy3KLuJD,2014,0.603,0.517,6,-8.504,0.0235,0.695,0.000003,0.744,0.327,96.181,"[rock, classic uk pop, pop]"
1,1y0U0HAe5QfTRzOsz74bOt,My Foolish Heart,25,166080,0,338mC0yGyX0C9of8QMJ5hK,1950-01-01,0.313,0.116,0,-12.645,0.0319,0.953,0.331000,0.161,0.255,74.071,"[rock, classic uk pop, pop]"


In [20]:
# for clustering
tracks_only_features = tracks[['danceability','key','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']].copy()

In [21]:
# for determining the genre that characterizes the group
tracks_only_genres = tracks[['artists_genre']]

In [22]:
tracks_only_features.head(2)

,danceability,key,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.603,6,0.517,-8.504,0.0235,0.695,0.000003,0.744,0.327,96.181
1,0.313,0,0.116,-12.645,0.0319,0.953,0.331000,0.161,0.255,74.071


## Clustering: pierwsza próba
Zgodnie z założeniami, korzystamy z metody K średnich.
Na razie zapoznajemy się z tym, jak ogólnie będzie wyglądało przypisanie gatunków dla piosenek.
Optymalna liczba grup oraz parametry modelu grupowania zostaną dobrane później, na razie sprawdzane jest, czy da się uzyskać jakiekolwiek wyniki

In [23]:
number_of_clusters = 1000

In [24]:
X = np.array(tracks_only_features).astype(float)
model = KMeans(n_clusters=number_of_clusters)
clusters_for_tracks = model.fit_predict(X)

##Ustalenie gatunku,który odpowiada grupie
Na podstawie gatunków, w których artyści tworzą, za pomocą heurystyk, jaki gatunek występuje w grupie najczęściej, określamy gatunek całej grupy.

In [25]:
artists_genres = tracks_only_genres.values.tolist()

In [26]:
genres_per_cluster = []
for i in range(number_of_clusters):
  genres_per_cluster.append([])

Dla każdego utworu dostaliśmy listę gatunków, w których tworzy jej autor

In [27]:
print(artists_genres[0][0])

['rock', 'classic uk pop', 'pop']


In [28]:
for i in range(len(clusters_for_tracks)):
  genres = artists_genres[i][0]
  for genre in genres:
    # if (genre != 'pop'):
    #   genres_per_cluster[clusters_for_tracks[i]].append(genre)
    genres_per_cluster[clusters_for_tracks[i]].append(genre)
  

Dla każdej grupy mamy przypisane wszystkie wystąpienia gatunkow u autorów piosenek z tej grupy

In [29]:
print(genres_per_cluster[0])

['rock', 'acoustic', 'pop', 'r&b', 'rock', 'cosmic american', 'country', 'double drumming', 'acoustic', 'jam band', 'anime score', 'classic anime', 'classic j-pop', 'pop', 'j-poprock', 'latin', 'pop', 'pop', 'pop', 'entehno', 'classical', 'greek pop', 'laiko', 'rebetiko', 'rock', 'metal', 'classic canadian rock', 'progressive rock', 'rock', 'acoustic', 'metal', 'arkansas country', 'country', 'classic j-pop', 'eurobeat', 'pop', 'eurodance', 'pop', 'german pop', 'dance', 'classic swedish pop', 'danspunk', 'traditional_folk', 'svensk progg', 'swedish jazz', 'visor', 'easy_listening', 'classical', 'jazz', 'country', 'country', 'argentine indie', 'argentine rock', 'latin', 'rock nacional', 'rock', 'progressive rock', 'rock', 'progressive rock', 'latin', 'bolero mexicano', 'rap', 'easy_listening', 'blues', 'jazz', 'easy_listening', 'blues', 'jazz', 'metal', 'rock', 'brazilian rock', 'jazz', 'pop rock brasileiro', 'rock gaucho', 'rock nacional brasileiro', 'rock', 'metal', 'irish rock', 'cana

In [30]:
def most_frequent(List):
  return max(set(List), key = List.count)

In [31]:
clusters_genres = []
for cluster in genres_per_cluster:
  clusters_genres.append(most_frequent(cluster))

Gatunek, występujący najczęściej w grupie określa gatunek grupy

In [32]:
print(len(set(clusters_genres)))
print(set(clusters_genres))

12
{'classic swedish pop', 'latin', 'dance', 'brass ensemble', 'rock', 'r&b', 'traditional_folk', 'country', 'jazz', 'acoustic', 'easy_listening', 'pop'}


## Wyniki
Nie najlepsze: bardzo niezbalansowane

In [33]:
tracks_genres=[]
for cluster in clusters_for_tracks:
  tracks_genres.append(clusters_genres[cluster])
tracks_with_genres = tracks.assign(genre=tracks_genres)

In [34]:
tracks_with_genres.head(5)

,id,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_genre,genre
0,48SFtLr5URCI97X2Ynfdnc,Par Avion (Live) ( 2014 - Remaster) - Live; 20...,0,291227,0,2yTUYhIf8fxptTIy3KLuJD,2014,0.603,0.517,6,-8.504,0.0235,0.6950,0.000003,0.7440,0.327,96.181,"[rock, classic uk pop, pop]",pop
1,1y0U0HAe5QfTRzOsz74bOt,My Foolish Heart,25,166080,0,338mC0yGyX0C9of8QMJ5hK,1950-01-01,0.313,0.116,0,-12.645,0.0319,0.9530,0.331000,0.1610,0.255,74.071,"[rock, classic uk pop, pop]",pop
2,7ij5kN8jwXr8fZD54M0xb6,Aleni Aleni,51,235974,0,48CUA59SDed3IdCctKndud,2015,0.684,0.839,4,-6.457,0.0658,0.1200,0.000000,0.3540,0.580,128.051,"[rock, classic uk pop, pop]",pop
3,6tM2QQSXvoOAEfH0cARToU,A Contraluz,28,230560,0,2TkVZG5xW47ZC0227ftg3q,2009-10-28,0.439,0.709,2,-6.803,0.0339,0.0584,0.000000,0.0803,0.495,155.018,"[rock, classic uk pop, pop]",pop
4,6UIcN1tiiGdd7oMMzNvyaP,Time,53,297502,0,7lbSsjYACZHn1MSDXPxNF2,2020-05-08,0.479,0.478,5,-8.884,0.1100,0.0601,0.000007,0.0813,0.117,100.040,"[rock, classic uk pop, pop]",pop


In [ ]:
tracks_with_genres.groupby('genre').count()

##Czy da sie to klasyfikować?


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [54]:
#print(tracks_only_features.values)
print(tracks_genres)

['pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'latin', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'easy_listening', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'latin', 'pop', 'pop', 'rock', 'rock', 'pop', 'pop', 'latin', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'latin', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'rock', 'pop', 'rock', 'rock', 'pop', 'pop', 'pop', 'rock', 'rock', 'rock', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'pop', 'roc

In [55]:
X = tracks_only_features.values
y = tracks_genres
# Split dataset into random train and test subsets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Standardize features by removing mean and scaling to unit variance:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



In [61]:
# Use the KNN classifier to fit data:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)



KNeighborsClassifier()

In [62]:
# Predict y data with classifier: 
y_predict = classifier.predict(X_test)

# Print results: 
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))
classifier.score(X_test, y_test)

[[    1     0     0     0     0     0     4     0     0     0]
 [    0     0     0     0     0     1     0     0     0     0]
 [    0     0     0     0     0     0    20     0     0     0]
 [    0     0     0     6     0     3    83     0     6     0]
 [    0     0     0     0     3     3    16     0     4     0]
 [    0     0     0     3     2    32   401     0    32     3]
 [    0     0     0    16     3    84 12762     0   495     0]
 [    0     0     0     0     0     1    16     2     0     0]
 [    0     0     0     9     4    34  2101     0   299     1]
 [    0     0     0     1     0     3    20     0     3     2]]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

           acoustic       1.00      0.20      0.33         5
classic swedish pop       0.00      0.00      0.00         1
            country       0.00      0.00      0.00        20
     easy_listening       0.17      0.06      0.09        98
               jazz       0.25      0.12      0.16        26
              latin       0.20      0.07      0.10       473
                pop       0.83      0.96      0.89     13360
                r&b       1.00      0.11      0.19        19
               rock       0.36      0.12      0.18      2448
   traditional_folk       0.33      0.07      0.11        29

           accuracy                           0.80     16479
          macro avg       0.41      0.17      0.21     16479
       weighted avg       0.73      0.80      0.75     16479



0.7953759330056436